The algorithm has to detect the lane markings
There are other techniques to achievw the final goals (edge detection kernel , transformations etc)

Canny Edge Detector Kernel :
    
it can detect edges like laplacian edge detector , more efficient than laplacian
optimal cuz every edge is detected only once

1: applying gaussian blue to reduce noise  (kernel size (5,5))
    noise is basically false edge detection
    so to remove noise , we must smoothen the image , just what gaussian blur does
    
2:find the intensity gradient of the image ( we have to use the kernel again - Sobel kernel)
    
3:non-maximum suppression :it removes the pixels that are not considered to be part of an edge so only edges with higher probability will remain . edge = object

4:hysteresis:
Pixel gradients higher than the upper treshold are accepted as an edge -> means if the difference (pixel intensities -> pixel gradients) between the neighbouring pixels is high , then theres a high prob that theres an edge . (100 ,255) -> edge
if the pixel intensity is between lower and higher pixels then it will be accepted only if it is connected to a pixel that is above the upper treshold



In [1]:
import cv2
import numpy as np
import os

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
def draw_the_lines(image , lines):
    lines_image = np.zeros((image.shape[0] , image.shape[1] , 3) , dtype = np.uint8)
    
    for line in lines: #x1,y1 = starting point , x2,y2 = end points
        for x1,y1 , x2,y2 in line:                      #B  G R
            cv2.line(lines_image , (x1,y1) , (x2,y2) , (0,0,255) , thickness = 3)
    
#     To merge the line                              0.8 and 1 is transparency
#     print(image.shape)
    image = cv2.cvtColor(image , cv2.COLOR_GRAY2RGB)
#     print(image.shape)
    image_with_lines = cv2.addWeighted(image , 0.8 , lines_image , 1 , 0.0)
    
    
    return image_with_lines

def region_of_interest(image , region_points):
# there is no need for the whole image : we just want to consider the lanes , so:
# we will create a mask and apply logical AND operator on the pixels 
    
    mask = np.zeros_like(image) # array filled with 0(black) of dimensionality of image
    
    cv2.fillPoly(mask , region_points , 255) # fill the region_points with (255)white
    
    mask = cv2.bitwise_and(image , mask)
    return mask

def get_detected_lanes(image):
    (height , width) = (image.shape[0] , image.shape[1])
#     print(height , width)
          
#     image = np.float32(image)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)    
    canny_image = cv2.Canny(image,100,120)
    
    # we are going to define 3 point (left , middle , right) (polygon)
    region_of_interest_vertices = [(0 , height) , (width/2 , height*0.65) , (width , height)]    
    region_of_interest_vertices = np.array([region_of_interest_vertices] , np.int32)
#     print(region_of_interest_vertices.shape)
    cropped_image = region_of_interest(canny_image , region_of_interest_vertices)    
    
    #Hough Algorithm
    
    lines = cv2.HoughLinesP(cropped_image , rho = 2 , theta = np.pi/180 , threshold = 50 , lines = np.array([]) , minLineLength = 40 , maxLineGap = 150)
    
#     rho = r = distance from origin to line , the resolution of the parameter r in pixels
#     theta = the resolution of the parameter degree(angle) in pixels
#     threshold = the minimum no of intersections to detect a line
#     minLineLength = 40 -> line length shorter than 40 are rejected
#     maxLineGap -> max allowed gap between 2 lines
    
    image_with_lines = draw_the_lines(image , lines)
    
    return image_with_lines
    
    
        
# os.chdir('F:/python/Neural_Networks/Lane-Detection-using-OpenCV')
# abs_path = os.path.abspath('F://python/Neural_Networks/Lane-Detection-using-OpenCV/abc.mp4')


video = cv2.VideoCapture('abc.mp4')

while video.isOpened():
    
    is_grabbed , frame = video.read() # is_grabbed = true if frame is detected
    
    if not is_grabbed : # is_grabbed will be false when video will be finished
        break
#     print(frame.shape)
    frame = get_detected_lanes(frame)
    
    cv2.imshow("Lane Detection Video" , frame)
    cv2.waitKey(15)

    
video.release()    
cv2.destroyAllWindows()


We have deal with the lines in the images . So we will apply hough algorithm . So after applying th edge detection kernel , we can find lines in the transformed image . 

[[0 1]
 [9 8]
 [5 5]]
